# Practica 3: Jaime Héctor y Álvaro Sánchez

## Preprocesamiento de datos

En primer lugar, es necesario realizar un preprocesamiento de los datos para garantizar que estén limpios, completos y en un formato adecuado que permita al modelo de machine learning aprender de manera eficiente. Para ello se deben evitar los valores nulos, normalizar las variables numéricas y codificar las variables categóricas.

In [2]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Cargar los datos en una variable
datos_test = pd.read_csv("public_test.csv")  # Fichero CSV con datos de test
datos_train = pd.read_csv("public_train.csv")  # Fichero CSV con datos de entrenamiento

# Separar la información del resultado ovariable objetivo
x_train = datos_train.drop(columns="ccs")  # 'ccs' es la columna objetivo
y_train = datos_train["ccs"]
x_test = datos_test

# Iniciar transmormadores para adaptar los datos al modelo
scaler = StandardScaler()
numeric_imputer = SimpleImputer(strategy="mean")  # Nulo numérico --> Media del resto
categorical_imputer = SimpleImputer(strategy="most_frequent")  # Nulo categórico --> Moda del resto
oh_encoder = OneHotEncoder(sparse_output=False)  # One-hot encoding: Codificar variables categóricas

# Transformador numérico (Eliminar nulos + normalizar)
numeric_transformer = Pipeline(steps=[
    ("imputer", numeric_imputer),
    ("scaler", scaler)
])

# Transformador categórico (Eliminar nulos + codificar)
categorical_transformer = Pipeline(steps=[
    ("imputer", categorical_imputer),
    ("encoder", oh_encoder)
])

# Agrupar las deiferentes columnas relevantes de los datos
desc_columns = [col for col in x_train.columns if col.startswith('desc_')]
fgp_columns = [col for col in x_train.columns if col.startswith('fgp_')]
adduct_column = ["adduct"]  # 'adduct' es categórica 

# Transformar usando ColumnTransformer, para elegir a que columnas aplicar cada tipo de transformación
transformer = ColumnTransformer(
    transformers=[
        ("numeric", numeric_transformer, desc_columns + fgp_columns),  # Numéricos
        ("categorical", categorical_transformer, adduct_column)  # Categóricos
    ]
)

# Aplicar los cambios realizados en los datos y mantenerlos como DataFrame
def preprocess_data(x):
    transformed = transformer.fit_transform(x)
    return pd.DataFrame(transformed, index=x.index)

x_train = preprocess_data(x_train)
x_test = preprocess_data(x_test)


## Entrenamiento y estimación del error

In [38]:
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import median_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# Dividir los datos: 70% entrenamiento, 15% validación, 15% test
x_train_final, x_temp, y_train_final, y_temp = train_test_split(x_train, y_train, test_size=0.3, random_state=42)
x_val, x_test_train, y_val, y_test_train = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Bucle interno para selección de hiperparámetros
def fit_model(x_train_final, y_train_final, model):
    inner_kfold = KFold(n_splits=5, shuffle = True,random_state=42)
    maes = []
    for train_indices, val_indices in inner_kfold.split(x_train_final):
        x_train = x_train_final.iloc[train_indices]
        y_train = y_train_final.iloc[train_indices]
        x_val = x_train_final.iloc[val_indices]
        y_val = y_train_final.iloc[val_indices]

        model.fit(x_train, y_train)
        y_val_pred = model.predict(x_val)
        maes.append(median_absolute_error(y_val, y_val_pred))

    avg_mae = np.mean(maes)

    return avg_mae

# Modelos a comparar
models = [
    ("Linear Regression", LinearRegression()),
    ("Random Forest", RandomForestRegressor(5))
]

# Bucle externo para validación cruzada
outer_kfold = KFold(n_splits=5, shuffle = True ,random_state=42)
results = []
fold = 1
for train_val_indices, test_indices in outer_kfold.split(x_train, y_train):
    x_train_val = x_train.iloc[train_val_indices]
    y_train_val = y_train.iloc[train_val_indices]
    x_test_fold = x_train.iloc[test_indices]
    y_test_fold = y_train.iloc[test_indices]

    print(f"Fold {fold}")
    for name, model in models:
        print(f"Evaluando modelo: {name}")
        avg_mae = fit_model(x_train_val, y_train_val, model)

        model.fit(x_train_val, y_train_val)
        y_test_pred = model.predict(x_test_fold)
        mae_test = median_absolute_error(y_test_fold, y_test_pred)
        print(f"MAE en el conjunto  (Fold {fold}): {mae_test}")

        results.append((name, fold, avg_mae, mae_test))

    fold += 1


Fold 1
Evaluando modelo: Linear Regression
MAE en el conjunto  (Fold 1): 3.7602733086581424
Evaluando modelo: Random Forest
MAE en el conjunto  (Fold 1): 3.516498000000013
Fold 2
Evaluando modelo: Linear Regression
MAE en el conjunto  (Fold 2): 3.895107819362252
Evaluando modelo: Random Forest
MAE en el conjunto  (Fold 2): 3.453996999999987
Fold 3
Evaluando modelo: Linear Regression
MAE en el conjunto  (Fold 3): 3.8127746781627536
Evaluando modelo: Random Forest
MAE en el conjunto  (Fold 3): 3.3357790000000023
Fold 4
Evaluando modelo: Linear Regression
MAE en el conjunto  (Fold 4): 3.8553671912998624
Evaluando modelo: Random Forest
MAE en el conjunto  (Fold 4): 3.3449979999999755
Fold 5
Evaluando modelo: Linear Regression
MAE en el conjunto  (Fold 5): 3.837829234451391
Evaluando modelo: Random Forest
MAE en el conjunto  (Fold 5): 3.5383330000000086


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

# Separar los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba interno)
x_train_final, x_test_train, y_train_final, y_test_train = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42
)

# Separar el conjunto de entrenamiento en entrenamiento y validación (75% entrenamiento, 25% validación)
x_train_final, x_val_train, y_train_final, y_val_train = train_test_split(
    x_train_final, y_train_final, test_size=0.25, random_state=42
)

# Verificar las dimensiones de los conjuntos
print("Dimensiones de los conjuntos:")
print("x_train_final:", x_train_final.shape)
print("x_val_train:", x_val_train.shape)
print("x_test_train:", x_test_train.shape)

# Crear y entrenar el modelo (Random Forest Regressor como ejemplo)
model = RandomForestRegressor(random_state=42, n_estimators=100)
model.fit(x_train_final, y_train_final)

# Predecir valores en los conjuntos de validación y prueba interno
y_val_pred = model.predict(x_val_train)
y_test_pred = model.predict(x_test_train)

# Calcular el MAE para validación y prueba interno
mae_val = mean_absolute_error(y_val_train, y_val_pred)
mae_test = mean_absolute_error(y_test_train, y_test_pred)

print("MAE en conjunto de validación:", mae_val)
print("MAE en conjunto de prueba interno:", mae_test)

# Si el modelo tiene buen rendimiento, aplicarlo al conjunto de prueba real
final_predictions = model.predict(x_test)

# Guardar las predicciones en un archivo CSV
import pandas as pd
output = pd.DataFrame(final_predictions, columns=["ccs_pred"])
output.to_csv("test_preds.csv", index=False, header=False)

## Generación de predicciones

## Conclusiones